In [1]:
from VariationalBayes import ScalarParam, ModelParamsDict, VectorParam
from VariationalBayes.NormalParams import MVNParam, UVNParam
from VariationalBayes.GammaParams import GammaParam

from autograd import grad, hessian, jacobian, hessian_vector_product
import autograd.numpy as np

import VariationalBayes as vb


def ApproxEq(v1, v2, tol=1e-8):
    return np.max(np.abs(v1 - v2)) < tol

In [2]:
vec = np.array([0.5, 2.5])
for lb in [ -float("inf"), 0.2]:
    for ub in [ float("inf"), 2.7]:
        vec_free = vb.Parameters.unconstrain_vector(vec, lb=lb, ub=ub)
        vec_test = vb.Parameters.constrain(vec_free, lb=lb, ub=ub)
        assert ApproxEq(vec, vec_test)

In [3]:
mat = np.full(4, 0.2).reshape(2, 2) + np.eye(2)
vec = vb.Parameters.vectorize_matrix(mat)
assert ApproxEq(mat, vb.Parameters.unvectorize_matrix(vec))

In [5]:
import math

mat = np.full(4, 0.2).reshape(2, 2) + np.eye(2)
vec = vb.Parameters.vectorize_matrix(mat)

VecGrad = jacobian(vb.Parameters.vectorize_matrix)
VecGrad(mat)

def unvectorize_matrix(vec):
    mat_size = int(0.5 * (math.sqrt(1 + 8 * vec.size) - 1))
    if mat_size * (mat_size + 1) / 2 != vec.size: \
        raise ValueError('Vector is an impossible size')
    nums = np.array([])
    for k1 in range(mat_size):
        for k2 in range(mat_size):
            nums.append(vec[SymIndex(k1, k2)])
    return np.array(nums).reshape(mat_size, mat_size)

assert ApproxEq(mat, unvectorize_matrix(vec))

UnvecGrad = jacobian(unvectorize_matrix)
print VecGrad(vec)



AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [14]:
# works
def MyFun(vec):
    return np.concatenate((vec[0:2], vec[2:4])).reshape(2,2)

vec = np.array([1., 2., 3., 4.])
MyFun(vec)

MyFunJacobian = jacobian(MyFun)
MyFunJacobian(vec)

array([[[ 1.,  0.,  0.,  0.],
        [ 0.,  1.,  0.,  0.]],

       [[ 0.,  0.,  1.,  0.],
        [ 0.,  0.,  0.,  1.]]])

In [28]:
# Does not work
def MyFun(vec):
    foo = vec[0:2]
    foo = np.append(foo, vec[2:4])
    return foo

vec = np.array([1., 2., 3., 4.])
MyFun(vec)

MyFunJacobian = jacobian(MyFun)
# MyFunJacobian(vec)